In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('../green cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = (df.groupby('permno')['ret_excess'].shift(-1) )

In [44]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Assuming df is already loaded with columns: datadate, permno, mom12m, mve, bm

# Convert datadate to datetime if not already
df['datadate'] = pd.to_datetime(df['datadate'])

# Define training window (15 years before January 2020)
train_start = pd.to_datetime('2005-01-31')
train_end = pd.to_datetime('2019-12-31')
predict_date = pd.to_datetime('2020-01-31')

# Filter training data
train_df = df[(df['datadate'] >= train_start) & (df['datadate'] <= train_end)].copy()

# Create binary target based on top quintile (top 20%) of returns
# Group by date and calculate quintile thresholds
# train_df['target'] = train_df.groupby('datadate')['ret_fwd_1'].transform(
#     lambda x: (x >= x.quantile(0.90)).astype(int)
# )
train_df['target'] = (train_df['ret_fwd_1'] > 0).astype(int)

# Remove rows with missing values
train_df = train_df.dropna(subset=['mom12m', 'mve', 'bm', 'target'])

# Prepare features
features = ['mom12m', 'mve', 'bm']
X_train = train_df[features]
y_train = train_df['target']

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Train logistic regression
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Get prediction data for January 2020
predict_df = df[df['datadate'] == predict_date].copy()

predict_df = predict_df.dropna(subset=['mom12m', 'mve', 'bm'])

# Prepare features for prediction
X_predict = predict_df[features]
X_predict_scaled = scaler.transform(X_predict)

# Generate buy signals (probability of positive return)
predict_df['buy_probability'] = log_reg.predict_proba(X_predict_scaled)[:, 1]
predict_df['buy_signal'] = log_reg.predict(X_predict_scaled)

# Sort by buy probability (highest first)
predict_df = predict_df.sort_values('buy_probability', ascending=False)

# Display probability distribution
print(f"Training samples: {len(train_df)}")
print(f"Training accuracy: {log_reg.score(X_train_scaled, y_train):.4f}")
print(f"\nTotal stocks evaluated in Jan 2020: {len(predict_df)}")
print(f"\nBuy probability statistics:")
print(predict_df['buy_probability'].describe())
print(f"\nStocks with buy probability >= 0.75: {len(predict_df[predict_df['buy_probability'] >= 0.75])}")

# If no stocks meet 0.75 threshold, show top percentile
threshold = 0.75
high_confidence_buys = predict_df[predict_df['buy_probability'] >= threshold].copy()

if len(high_confidence_buys) == 0:
    print(f"\nNo stocks meet the 0.75 threshold.")
    print(f"Consider using a lower threshold or selecting top N stocks.")
    print(f"\nTop 50 stocks by buy probability:")
    top_50 = predict_df.nlargest(50, 'buy_probability')
    bottom_50 = predict_df.nsmallest(50, 'buy_probability')
    # 3. Combine the two dataframes
    top_and_bottom_50 = pd.concat([top_50, bottom_50])
    
    print(top_and_bottom_50[['permno', 'comnam', 'mom12m', 'mve', 'bm', 'buy_probability']].to_string())
    signals_jan2020 = top_and_bottom_50[['permno', 'comnam', 'datadate', 'mom12m', 'mve', 'bm', 'buy_probability']]
else:
    print(f"\nHigh confidence buy signals (probability >= {threshold}):")
    print(high_confidence_buys[['permno', 'comnam', 'mom12m', 'mve', 'bm', 'buy_probability']].to_string())
    buy_signals_jan2020 = high_confidence_buys[['permno', 'comnam', 'datadate', 'mom12m', 'mve', 'bm', 'buy_probability']]

Training samples: 89793
Training accuracy: 0.5590

Total stocks evaluated in Jan 2020: 497

Buy probability statistics:
count    497.000000
mean       0.558957
std        0.009452
min        0.532203
25%        0.553408
50%        0.558889
75%        0.565185
max        0.582578
Name: buy_probability, dtype: float64

Stocks with buy probability >= 0.75: 0

No stocks meet the 0.75 threshold.
Consider using a lower threshold or selecting top N stocks.

Top 50 stocks by buy probability:
        permno                           comnam    mom12m       mve        bm  buy_probability
30921    14593                        APPLE INC  1.986466  1.857785 -0.874105         0.582578
248109   86580                      NVIDIA CORP  1.592618  1.798087 -0.851453         0.580572
264584   91233                   MASTERCARD INC  1.278524  1.857785 -1.094125         0.580365
1145     10107                   MICROSOFT CORP  1.082318  1.857785 -0.873725         0.579112
23644    13407                     F

In [45]:
signals_jan2020

,permno,comnam,datadate,mom12m,mve,bm,buy_probability
30921,14593,APPLE INC,2020-01-31,1.986466,1.857785,-0.874105,0.582578
248109,86580,NVIDIA CORP,2020-01-31,1.592618,1.798087,-0.851453,0.580572
264584,91233,MASTERCARD INC,2020-01-31,1.278524,1.857785,-1.094125,0.580365
1145,10107,MICROSOFT CORP,2020-01-31,1.082318,1.857785,-0.873725,0.579112
23644,13407,FACEBOOK INC,2020-01-31,1.195430,1.857785,-0.492363,0.578680
...,...,...,...,...,...,...,...
119769,37584,FRANKLIN RESOURCES INC,2020-01-31,-1.364363,-0.840055,0.735571,0.545608
64625,21186,WESTROCK CO,2020-01-31,-0.608374,-1.012452,1.405914,0.545700
161147,52230,ROBERT HALF INTERNATIONAL INC,2020-01-31,-0.973644,-1.451326,-0.702304,0.545809
107380,27991,PINNACLE WEST CAPITAL CORP,2020-01-31,-0.883338,-1.113003,0.490640,0.545977


## Then we run nodewise on the buys

In [46]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')


def naive_nodewise_regression(Y_star, lambda_grid=None):
    """
    Implements Naive Nodewise Regression (Section 5.1.2).
    Uses GIC (Generalized Information Criterion) as in the paper.
    
    Parameters:
    -----------
    Y_star : np.ndarray, shape (n, p)
        Demeaned returns matrix (time x assets)
    lambda_grid : list or None
        Grid of lambda values to try. If None, creates default grid.
    
    Returns:
    --------
    Theta_hat : np.ndarray, shape (p, p)
        Estimated precision matrix
    """
    n, p = Y_star.shape

    # Initialize matrices
    Theta_hat = np.zeros((p, p))
    tau_squared = np.zeros(p)
    
    # Create lambda grid if not provided
    if lambda_grid is None:
        lambda_grid = np.logspace(-3, 1, 50)
    
    # For each asset j
    for j in range(p):
        # Step 1: Get y_j (target) and Y_{-j} (predictors)
        y_j = Y_star[:, j]
        Y_minus_j = np.delete(Y_star, j, axis=1)
        
        # Step 2-3: Estimate gamma_j using Lasso with GIC
        # GIC(λ) = log(σ²_λ) + |S_λ| * (log(p-1) / n) * log(log(n))
        best_gic = np.inf
        best_lambda = lambda_grid[0]
        best_gamma = None
        best_ssr = None
        
        for lam in lambda_grid:
            lasso = Lasso(alpha=2*lam, fit_intercept=False, max_iter=10000)
            lasso.fit(Y_minus_j, y_j)
            gamma_j = lasso.coef_
            
            # Compute SSR and number of non-zero coefficients
            residuals = y_j - Y_minus_j @ gamma_j
            ssr = np.sum(residuals ** 2)
            sigma_sq_lambda = ssr / n
            q_lambda = np.sum(np.abs(gamma_j) > 1e-8)
            
            # Compute GIC
            if sigma_sq_lambda > 1e-10: # Check for non-zero variance
                # GIC formula from paper
                gic = np.log(sigma_sq_lambda) + q_lambda * (np.log(p) / n) * np.log(np.log(n))
            else:
                gic = np.inf
            
            if gic < best_gic:
                best_gic = gic
                best_lambda = lam
                best_gamma = gamma_j.copy()
                best_ssr = ssr
        
        gamma_j_star = best_gamma
        

        tau_squared[j] = best_ssr / n + best_lambda * np.sum(np.abs(gamma_j_star))

        # [cite_start]Step 5: Form the j-th row of Theta_hat [cite: 579, 543-547]
        Theta_hat[j, j] = 1 / tau_squared[j]
        off_diag = -gamma_j_star / tau_squared[j]
        Theta_hat[j, :j] = off_diag[:j]
        Theta_hat[j, j+1:] = off_diag[j:]
    
    # Step 6: Symmetrize
    Theta_hat_sym = (Theta_hat + Theta_hat.T) / 2
    
    return Theta_hat_sym


def gmv_weights(Theta_hat):
    """
    Compute Global Minimum Variance (GMV) portfolio weights (Section 6.1).
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # w* = (Θ 1_p) / (1_p' Θ 1_p)
    numerator = Theta_hat @ ones_p
    denominator = ones_p @ Theta_hat @ ones_p
    
    if np.abs(denominator) < 1e-10:
        # Fallback to equal weights if precision matrix is near-singular
        return ones_p / p
    
    w_star = numerator / denominator
    
    return w_star


def compute_portfolio_metrics(returns, weights):
    """
    Compute portfolio return, variance, and Sharpe ratio.
    """

    portfolio_returns = returns @ weights
    mean_return = np.mean(portfolio_returns)
    variance = np.var(portfolio_returns, ddof=1)
    sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
    
    return {
        'return': mean_return,
        'variance': variance,
        'sharpe_ratio': sharpe_ratio
    }


def backtest_nodewise_gmv(df, 
                          test_start_date='2000-01-31', 
                          test_end_date='2003-12-31',
                          lookback_window=180,
                          transaction_cost=0.005,
                          verbose=True):
    """
    Backtest Nodewise + GMV strategy with monthly rebalancing,
    180-month rolling window, and NaN filtering as per the paper.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: permno, datadate, ret_fwd_1
    test_start_date : str
        First date for out-of-sample returns (format: 'YYYY-MM-DD')
    test_end_date : str
        Last date for out-of-sample returns (format: 'YYYY-MM-DD')
    lookback_window : int
        Number of months in rolling training window (default: 180)
    transaction_cost : float
        Proportional transaction cost (default: 0.005 = 50 bps)
    verbose : bool
        If True, prints detailed log at each time step.
    
    Returns:
    --------
    results_df : pd.DataFrame
        DataFrame with columns: date, portfolio_return, cumulative_return
    metrics : dict
        Overall performance metrics
    """
    # --- 1. Setup ---
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])
    
    # Get unique dates
    all_dates = sorted(df['datadate'].unique())
    
    # Convert test dates to datetime
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    # Find date indices
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    if test_start_idx < lookback_window:
        raise ValueError(f"Not enough data for lookback. Test start date {test_start_date} "
                         f"requires data back to {all_dates[test_start_idx - lookback_window]}, "
                         f"but only {test_start_idx} periods are available.")
    
    # Storage for results
    portfolio_returns = []
    portfolio_dates = []
    portfolio_weights_list = []
    portfolio_turnover_list = []
    portfolio_gross_returns = []
    
    # FIX 7: Use dictionary to track weights by permno (handles entry/exit)
    prev_weights_dict = {}  # Maps permno -> weight
    prev_oos_returns_dict = {}  # Maps permno -> return
    prev_gross_return = 0.0
    
    # --- 2. Rolling Window Backtest ---
    if verbose:
        print("="*60)
        print("STARTING BACKTEST")
        print("="*60)
        
    for t in range(test_start_idx, test_end_idx + 1):
        current_date = all_dates[t]
        
        # Define the lookback window
        window_start_date = all_dates[t - lookback_window]
        window_end_date = all_dates[t - 1]
        
        # Get training data for this window
        train_data = df[(df['datadate'] >= window_start_date) & 
                        (df['datadate'] <= window_end_date)]
        
        # Pivot to get returns matrix (time x assets)
        returns_pivot = train_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        
        # Reindex to ensure all dates are present
        window_dates = all_dates[t - lookback_window : t]
        returns_pivot = returns_pivot.reindex(index=window_dates)
        
        # Filter assets with any NaNs in this window
        # (This follows the paper's approach of requiring complete data)
        nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
        filtered_pivot = returns_pivot.drop(columns=nan_assets)
        
        current_assets = filtered_pivot.columns.tolist()
        Y = filtered_pivot.values
        n_train, p_current = Y.shape

        if verbose:
            print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                  f"Date: {current_date.strftime('%Y-%m-%d')}")
            print(f"  Window: {window_start_date.strftime('%Y-%m-%d')} to "
                  f"{window_end_date.strftime('%Y-%m-%d')}")
            print(f"  Assets: {p_current} with complete data")

        # Check for valid data
        if n_train < lookback_window or p_current < 2:
            if verbose:
                print(f"  ⚠ Insufficient data (n={n_train}, p={p_current}), using prev weights")
            new_weights_dict = prev_weights_dict.copy()
        else:
            try:
                # Demean the returns
                Y_bar = Y.mean(axis=0)
                Y_star = Y - Y_bar
                
                if verbose:
                    print(f"  Running Nodewise Regression...")
                Theta_hat = naive_nodewise_regression(Y_star)
                
                if verbose:
                    print(f"  Computing GMV weights...")
                w_star = gmv_weights(Theta_hat)
                
                # Create weights dictionary
                new_weights_dict = {asset: w_star[i] for i, asset in enumerate(current_assets)}
                
            except Exception as e:
                if verbose:
                    print(f"  ✗ Error: {e}")
                    print(f"  Using previous weights")
                new_weights_dict = prev_weights_dict.copy()

        # Normalize weights to sum to 1
        weight_sum = sum(new_weights_dict.values())
        if weight_sum > 1e-10:
            new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
        else:
            if verbose:
                print("  ⚠ Zero weight sum, using previous weights")
            new_weights_dict = prev_weights_dict.copy()
        
        # --- 3. OOS Returns & Transaction Costs ---
        
        # Get out-of-sample returns for current month
        oos_data = df[df['datadate'] == current_date]
        oos_returns_series = oos_data.set_index('permno')['ret_fwd_1']
        
        # FIX 8: Filter out NaN returns and create dictionary
        oos_returns_series = oos_returns_series.dropna()
        oos_returns_dict = oos_returns_series.to_dict()
        
        # Find common assets between weights and returns (both existing and non-NaN)
        common_assets = set(new_weights_dict.keys()) & set(oos_returns_dict.keys())
        
        if len(common_assets) == 0:
            if verbose:
                print("  ⚠ No common assets with valid returns, skipping period")
            continue
        
        # Filter to common assets and renormalize
        common_weights = {a: new_weights_dict[a] for a in common_assets}
        common_weight_sum = sum(common_weights.values())
        if common_weight_sum > 1e-10:
            common_weights = {k: v/common_weight_sum for k, v in common_weights.items()}
        else:
            if verbose:
                print("  ⚠ Zero weight sum after filtering, skipping period")
            continue
        
        # Compute gross portfolio return (all returns should be valid now)
        gross_return = sum(common_weights[a] * oos_returns_dict[a] for a in common_assets)
        
        # Sanity check
        if np.isnan(gross_return) or np.isinf(gross_return):
            if verbose:
                print(f"  ⚠ Invalid gross return: {gross_return}, skipping period")
            continue
        
        # FIX 9: Calculate transaction costs with proper weight adjustment
        if len(prev_weights_dict) > 0:
            # Get the universe of all assets (current + previous)
            all_traded_assets = set(common_weights.keys()) | set(prev_weights_dict.keys())
            
            # Adjust previous weights for all assets that were held
            # w+_{t,j} = w_{t,j} * (1 + r_{t,j}) / (1 + r_p,t)
            adjusted_prev = {}
            for asset in all_traded_assets:
                prev_w = prev_weights_dict.get(asset, 0.0)
                
                if asset in prev_oos_returns_dict:
                    prev_r = prev_oos_returns_dict[asset]
                    # Avoid division by zero
                    if prev_gross_return > -0.99:  # Allow for up to 99% loss
                        adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
                    else:
                        adjusted_prev[asset] = 0.0
                else:
                    # Asset wasn't in portfolio last period, so adjusted weight is 0
                    adjusted_prev[asset] = 0.0
            
            # Renormalize adjusted weights (only over assets that still exist)
            adj_sum = sum(adjusted_prev.get(a, 0.0) for a in common_weights.keys())
            if adj_sum > 1e-10:
                adjusted_prev_normalized = {k: adjusted_prev.get(k, 0.0)/adj_sum 
                                           for k in common_weights.keys()}
            else:
                adjusted_prev_normalized = {k: 0.0 for k in common_weights.keys()}
            
            # Turnover: sum over ALL assets (current and previous)
            # This captures: rebalancing existing positions + exiting old + entering new
            turnover = sum(abs(common_weights.get(a, 0.0) - adjusted_prev_normalized.get(a, 0.0)) 
                          for a in all_traded_assets)
            
            # Transaction cost: c * (1 + gross_return) * turnover
            tc = transaction_cost * (1 + gross_return) * turnover
        else:
            # First period: full turnover (entering all positions)
            turnover = sum(abs(w) for w in common_weights.values())
            tc = transaction_cost * turnover
        
        # Net return
        net_return = gross_return - tc
        
        # Store results
        portfolio_returns.append(net_return)
        portfolio_dates.append(current_date)
        portfolio_weights_list.append(common_weights.copy())
        portfolio_turnover_list.append(turnover)
        portfolio_gross_returns.append(gross_return)
        
        # Update previous values for next iteration
        prev_weights_dict = common_weights.copy()
        prev_oos_returns_dict = {a: oos_returns_dict[a] for a in common_assets}
        prev_gross_return = gross_return
        
        if verbose:
            print(f"  Gross: {gross_return:>8.5f} | Turnover: {turnover:>6.4f} | "
                  f"TC: {tc:>8.6f} | Net: {net_return:>8.5f}")

    if verbose:
        print("\n" + "="*60)
        print("BACKTEST COMPLETE")
        print("="*60)
    
    # --- 4. Compile Results ---
    results_df = pd.DataFrame({
        'date': portfolio_dates,
        'portfolio_return': portfolio_returns,
        'portfolio_gross_return': portfolio_gross_returns,
        'portfolio_weights': portfolio_weights_list,
        'portfolio_turnover': portfolio_turnover_list
    })
    results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
    
    # Compute overall metrics
    if len(portfolio_returns) > 0:
        mean_return = np.mean(portfolio_returns)
        variance = np.var(portfolio_returns, ddof=1)
        sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
        
        # Annualized metrics (monthly data)
        annual_return = mean_return * 12
        annual_volatility = np.sqrt(variance * 12)
        annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
        
        metrics = {
            'mean_return': mean_return,
            'variance': variance,
            'sharpe_ratio': sharpe_ratio,
            'annual_return': annual_return,
            'annual_volatility': annual_volatility,
            'annual_sharpe_ratio': annual_sharpe,
            'total_return': results_df['cumulative_return'].iloc[-1],
            'avg_turnover': np.mean(portfolio_turnover_list),
            'n_periods': len(portfolio_returns)
        }
    else:
        metrics = {
            'mean_return': 0,
            'variance': 0,
            'sharpe_ratio': 0,
            'annual_return': 0,
            'annual_volatility': 0,
            'annual_sharpe_ratio': 0,
            'total_return': 0,
            'avg_turnover': 0,
            'n_periods': 0
        }
    
    return results_df, metrics

In [47]:
signals = signals_jan2020['permno'].values

In [48]:
df_filtered = df[df['permno'].isin(signals)]

In [49]:
df_filtered

,datadate,permno,comnam,ncusip,shrcd,exchcd,siccd,industry,ticker,gvkey_x,...,BETA,betasq,rsq1,pricedelay,idiovol,year,mom6,indmom,industry_return,ret_fwd_1
413,1989-08-31,10104,ORACLE SYSTEMS CORP,68389X10,11,3,7370,BusEq,ORCL,12142,...,2.514465,3.743249,0.253685,1.274151,3.015250,1989,NaN,-1.543315,0.007978,0.253500
414,1989-09-30,10104,ORACLE SYSTEMS CORP,68389X10,11,3,7370,BusEq,ORCL,12142,...,2.312440,3.721503,0.250007,1.156890,3.033384,1989,NaN,-1.624952,-0.017305,-0.022673
415,1989-10-31,10104,ORACLE SYSTEMS CORP,68389X10,11,3,7370,BusEq,ORCL,12142,...,2.284937,3.830593,0.247356,1.603935,2.594976,1989,NaN,-1.575948,-0.048984,0.025358
416,1989-11-30,10104,ORACLE SYSTEMS CORP,68389X10,11,3,7370,BusEq,ORCL,12142,...,2.274334,3.776422,0.261054,1.271141,2.662753,1989,NaN,-1.129246,-0.026846,-0.032142
417,1989-12-31,10104,ORACLE SYSTEMS CORP,68389X10,11,3,7370,BusEq,ORCL,12142,...,2.442270,3.851240,0.269939,1.363018,2.580838,1989,NaN,-1.360810,-0.000186,-0.085914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267637,2024-08-31,92611,VISA INC,92826C83,11,1,7389,Other,V,179534,...,-0.353241,0.610957,0.312394,0.362197,-0.958429,2024,-0.049536,-0.076201,0.011544,-0.009138
267638,2024-09-30,92611,VISA INC,92826C83,11,1,7389,Other,V,179534,...,-0.405590,0.592915,0.300659,0.628347,-0.953926,2024,-0.044559,-0.202169,0.040203,0.050292
267639,2024-10-31,92611,VISA INC,92826C83,11,1,7389,Other,V,179534,...,-0.423735,0.553885,0.291829,0.259124,-0.898447,2024,-0.037020,0.202610,-0.011252,0.085081
267640,2024-11-30,92611,VISA INC,92826C83,11,1,7389,Other,V,179534,...,-0.460288,0.526650,0.295736,0.034290,-0.942907,2024,0.055990,0.573966,0.054156,-0.000653


In [50]:
results_df, metrics = backtest_nodewise_gmv(
    df_filtered,
    test_start_date='2020-01-31',  
    test_end_date='2024-11-30',   # Last date of testing period
    lookback_window=180,
    transaction_cost=0.001
)
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {metrics['annual_sharpe_ratio']:.4f}")

STARTING BACKTEST

[1/59] Date: 2020-01-31
  Window: 2005-01-31 to 2019-12-31
  Assets: 64 with complete data
  Running Nodewise Regression...
  Computing GMV weights...
  Gross: -0.09536 | Turnover: 1.1022 | TC: 0.001102 | Net: -0.09647

[2/59] Date: 2020-02-29
  Window: 2005-02-28 to 2020-01-31
  Assets: 64 with complete data
  Running Nodewise Regression...
  Computing GMV weights...
  Gross: -0.13569 | Turnover: 0.0413 | TC: 0.000036 | Net: -0.13572

[3/59] Date: 2020-03-31
  Window: 2005-03-31 to 2020-02-29
  Assets: 65 with complete data
  Running Nodewise Regression...
  Computing GMV weights...
  Gross:  0.10443 | Turnover: 0.1103 | TC: 0.000122 | Net:  0.10431

[4/59] Date: 2020-04-30
  Window: 2005-04-30 to 2020-03-31
  Assets: 64 with complete data
  Running Nodewise Regression...
  Computing GMV weights...
  Gross:  0.02459 | Turnover: 0.0835 | TC: 0.000086 | Net:  0.02450

[5/59] Date: 2020-05-31
  Window: 2005-05-31 to 2020-04-30
  Assets: 64 with complete data
  Running 

In [26]:
with pd.option_context("display.max_rows", None):
    print(results_df['portfolio_weights'][0])

{14593: 0.01052975625840318, 20482: 0.039572046044568605, 49154: 0.019066678127968802, 66181: 0.027849145732252462, 77702: 0.017757042438909694, 62092: 0.027084744321226022, 24205: 0.04509443291148409, 87055: 0.036342776321466395, 47896: 0.015566817031728335, 12060: 0.01041347427718078, 13856: 0.06238250671300938, 10145: 0.02987625426441523, 26403: 0.027715408205383936, 17830: 0.034560178098258856, 55976: 0.04349670627739796, 11308: 0.05697916336471494, 86580: -0.00885788615498957, 89525: 0.022063023067059775, 14008: 0.01919162927459498, 43449: 0.05346398171022968, 21178: 0.03238046755455746, 59328: 0.023424554306810567, 19393: 0.022221220102227067, 57665: 0.026048239314999676, 60097: 0.02971185542136708, 17478: 0.017907328151346612, 10696: 0.032919122735013406, 48725: 0.024453508809885758, 15579: 0.023456167885935023, 22752: 0.028301768429249672, 19561: 0.015237283601087153, 14702: 0.00828154509650046, 18163: 0.05594053634906363, 75510: 0.008090911436090096, 10104: 0.02564157755520339

In [27]:
metrics['total_return']

0.3820372201951068

In [28]:
metrics['variance']

0.002220899227160404

In [29]:
results_df['portfolio_turnover'].mean()

0.06628990703329528